# Training and forecast of ML models

In [53]:
import pandas as pd

In [196]:
ts60 = pd.read_csv('../VEOLIA/artifacts/timeseries_60min.csv', index_col=0, parse_dates=True)
# load60 = ts60['Diff Load Activa Total (60 minuto)'].dropna()
# ts60.head()
weather = pd.read_csv('../VEOLIA/data/Burgos_weather.csv')
df = ts60[['Diff Load Activa Total (60 minuto)','TEMPERATURA EXTERIOR (60 minuto)']]
# df.head()

df.head()

,Diff Load Activa Total (60 minuto),TEMPERATURA EXTERIOR (60 minuto)
datetime,,
2020-11-01 00:00:00,NaN,NaN
2020-11-01 01:00:00,NaN,NaN
2020-11-01 02:00:00,NaN,NaN
2020-11-01 03:00:00,NaN,NaN
2020-11-01 04:00:00,NaN,NaN


In [197]:
weather.head(1)

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,...,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1591747200,2020-06-10 00:00:00 +0000 UTC,7200,Burgos,42.3503,-3.7072,5.51,2.72,4.94,7.03,...,30,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n


In [198]:
weather['datetime'] = weather['dt_iso'].str[0:20]
weather = weather.set_index('datetime')
weather.index = pd.to_datetime(weather.index)

In [199]:
new_df = pd.merge(df, weather, on = "datetime", how = "inner")[['Diff Load Activa Total (60 minuto)', 'temp', 'humidity', 'wind_speed', 'pressure']].dropna()

new_df.head()
df=new_df

In [200]:
df['year'] = df.index.year
df['month'] = df.index.month
df['month_day'] = df.index.day
df['week_day'] = df.index.weekday
df['hour'] = df.index.hour


In [201]:
df.head(150)

,Diff Load Activa Total (60 minuto),temp,humidity,wind_speed,pressure,year,month,month_day,week_day,hour
datetime,,,,,,,,,,
2020-11-06 00:00:00,2600.0,11.09,80,7.7,1020,2020,11,6,4,0
2020-11-06 01:00:00,2610.0,10.67,82,6.2,1020,2020,11,6,4,1
2020-11-06 02:00:00,2570.0,11.39,79,6.2,1019,2020,11,6,4,2
2020-11-06 03:00:00,2580.0,11.48,78,5.7,1018,2020,11,6,4,3
2020-11-06 04:00:00,2600.0,11.65,77,8.2,1018,2020,11,6,4,4
...,...,...,...,...,...,...,...,...,...,...
2020-11-12 00:00:00,1310.0,7.90,87,2.1,1025,2020,11,12,3,0
2020-11-12 01:00:00,2120.0,7.22,91,0.5,1025,2020,11,12,3,1
2020-11-12 02:00:00,1520.0,7.18,92,0.5,1025,2020,11,12,3,2


In [202]:
df.shape

(5824, 10)

## Regression models for load forecasting

In [203]:
X = df[['month', 'week_day', 'hour', 'temp', 'wind_speed','pressure', 'humidity']]
y = df['Diff Load Activa Total (60 minuto)']

In [204]:
from sklearn.model_selection import train_test_split

# random_state=23 in order to be led to reproducible results
# split 75%, 25%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)
print(X_train)
print(y_train)
print(X_test)
print(y_test)

                     month  week_day  hour   temp  wind_speed  pressure  \
datetime                                                                  
2020-12-02 00:00:00     12         2     0   5.47        3.10      1021   
2021-05-23 10:00:00      5         6    10  11.46        4.63      1019   
2021-03-24 14:00:00      3         2    14  16.96        2.60      1023   
2021-04-07 06:00:00      4         2     6  -0.04        7.20      1019   
2021-02-24 20:00:00      2         2    20   8.96        1.00      1028   
...                    ...       ...   ...    ...         ...       ...   
2021-03-06 01:00:00      3         5     1   3.96        5.70      1021   
2021-01-25 06:00:00      1         0     6   6.03        4.60      1007   
2020-12-19 14:00:00     12         5    14   7.98        4.60      1018   
2021-05-21 07:00:00      5         4     7  11.09        0.51      1017   
2020-11-30 13:00:00     11         0    13  12.11        4.60      1024   

                     hum

### Decision Tree Regression

In [205]:
from sklearn import tree

tree_reg = tree.DecisionTreeRegressor()
tree_reg = tree_reg.fit(X_train, y_train)

y_train_pred_dec_tree = tree_reg.predict(X_train)
y_test_pred_dec_tree = tree_reg.predict(X_test)


In [206]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error



In [207]:
MSE_train3 = mean_squared_error(y_train, y_train_pred_dec_tree)
MSE_test3 = mean_squared_error(y_test, y_test_pred_dec_tree)

MAPE_train3 = mean_absolute_percentage_error(y_train, y_train_pred_dec_tree)
MAPE_test3 = mean_absolute_percentage_error(y_test, y_test_pred_dec_tree)


print(MSE_test3, round(MAPE_test3 * 100, 2),'%')

163800.4120879099 10.23 %


### Random Forest Regression

In [208]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor()
regr.fit(X_train, y_train)

y_train_pred_rf = regr.predict(X_train)
y_test_pred_rf = regr.predict(X_test)

In [209]:
MSE_train_rf = mean_squared_error(y_train, y_train_pred_rf)
MSE_test_rf = mean_squared_error(y_test, y_test_pred_rf)

MAPE_train_rf = mean_absolute_percentage_error(y_train, y_train_pred_rf)
MAPE_test_rf = mean_absolute_percentage_error(y_test, y_test_pred_rf)


print(MSE_test_rf, round(MAPE_test_rf * 100, 2),'%')

78697.95038460741 7.86 %


### Gradient Boosting Regression

In [210]:
from sklearn.ensemble import GradientBoostingRegressor

reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)

y_train_pred_gb = reg.predict(X_train)
y_test_pred_gb = reg.predict(X_test)

In [211]:
MSE_train_gb = mean_squared_error(y_train, y_train_pred_gb)
MSE_test_gb = mean_squared_error(y_test, y_test_pred_gb)

MAPE_train_gb = mean_absolute_percentage_error(y_train, y_train_pred_gb)
MAPE_test_gb = mean_absolute_percentage_error(y_test, y_test_pred_gb)


print(MSE_test_gb, round(MAPE_test_gb * 100, 2),'%')

109636.56149938608 9.51 %


### LightGBM

In [212]:
import lightgbm as lgb

#### Todo:
1) LightGBM models
2) xgboost model
3) try with more complete weather data
4) parameter tuning
5) cross_validation

In [213]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [214]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}



In [215]:

# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_test,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's l1: 432.266	valid_0's l2: 316565
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 419.748	valid_0's l2: 299694
[3]	valid_0's l1: 408.005	valid_0's l2: 284408
[4]	valid_0's l1: 396.935	valid_0's l2: 270422
[5]	valid_0's l1: 384.574	valid_0's l2: 255442
[6]	valid_0's l1: 373.603	valid_0's l2: 242586
[7]	valid_0's l1: 364.239	valid_0's l2: 231748
[8]	valid_0's l1: 354.29	valid_0's l2: 220892
[9]	valid_0's l1: 345.406	valid_0's l2: 210820
[10]	valid_0's l1: 338.733	valid_0's l2: 203053
[11]	valid_0's l1: 329.728	valid_0's l2: 194007
[12]	valid_0's l1: 322.004	valid_0's l2: 186130
[13]	valid_0's l1: 314.682	valid_0's l2: 178834
[14]	valid_0's l1: 307.757	valid_0's l2: 171841
[15]	valid_0's l1: 301.865	valid_0's l2: 165952
[16]	valid_0's l1: 296.291	valid_0's l2: 160429
[17]	valid_0's l1

In [216]:
# predict
lgbm_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
mape_test = mean_absolute_percentage_error(y_test, lgbm_pred)
print(f'The MAPE of prediction is: {mape_test}')

The MAPE of prediction is: 0.11161829555274834


### XGBoost

In [217]:
import xgboost

xgb = xgboost.XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [218]:
xgb_pred = xgb.predict(X_test)

xgb_mape = mean_absolute_percentage_error(y_test, xgb_pred)
print(f'The MAPE of prediction is: {xgb_mape}')

The MAPE of prediction is: 0.08177443892097085
